In [2]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import indicoio
import numpy as np

# В этой тетрадке мы будем собирать комментарии по больницам из нашего датасета.

## Сперва сделали парсер для сайта prodoctorov.ru

In [3]:
def get_comment_from_html(html_comment):
    html_comment = BeautifulSoup(html_comment, "lxml")
    for a in html_comment.findAll('a'):
        del a['href']
    html_comment = str(html_comment)
    html_comment = html_comment.replace("<a>", "")
    html_comment = html_comment.replace("</a>", "")
    
    if (html_comment.find("<p class=\"comment_plus\">") != -1):
        html_comment = html_comment[html_comment.find("<p class=\"comment_plus\">") + len("<p class=\"comment_plus\">"):]
        good = html_comment[:html_comment.find("</p>")]
        
        html_comment = html_comment[html_comment.find("<p class=\"comment_minus\">") + len("<p class=\"comment_minus\">"):]
        bad = html_comment[:html_comment.find("</p>")]
        
        html_comment = html_comment[html_comment.find("<p class=\"comment\">") + len("<p class=\"comment\">"):]
        neutral = html_comment[:html_comment.find("</p>")]
        
        html_comment = good + ' ' + bad + ' ' + neutral
    else:
        html_comment = str(html_comment)    
        html_comment = html_comment[html_comment.find("itemprop=\"reviewBody\">") + len("itemprop=\"reviewBody\">"):]
        html_comment = html_comment[:html_comment.find("</p>")]
    
   
    return html_comment

In [4]:
def get_all_comments(link):
    source_code = requests.get(link)
    source_code.encoding = 'utf-8'
    print(source_code)
    soup =  BeautifulSoup(source_code.text, 'lxml')
    inf1 = soup.find_all('tr', itemprop="review")
    comments = []
    for ch in inf1:
        comments.append(get_comment_from_html(str(ch)))
    return comments

In [6]:
d = pd.read_csv('lpu.csv', encoding = "utf-8")

In [7]:
d

,Unnamed: 0,1,3,4,5,6,adress
0,25,ЗАО «Группа компаний «Медси»,2.813,4.000,3.125,"1,375","2-й Тверской-Ямской пер., д. 10; м. «Маяковская»"
1,59,ООО «Управляющая компания «Медсервис»,3.417,3.333,3.500,"1,533","Мичуринский пр-т, д. 6"
2,22,ООО «Дирекция» (Поликлиника РУ),3.867,3.600,3.533,"2,071","Романов пер., д. 2; м. «Библиотека им. Ленина»"
3,29,ОАО «Медицина»,4.344,4.438,3.644,"3,143","ул. Сущевский вал, д. 12"
4,16,ООО «Клиника ЛМС» (Будь здоров),3.875,4.063,3.656,"2,563","ул. Тимура Фрунзе, 15/1; м. «Парк Культуры»"
5,35,ООО «Клиника ЛМС» (Будь здоров),3.929,4.143,3.679,"2,714","ул. Усачева, д. 33, стр. 3"
6,24,ЗАО «КООПВНЕШТОРГ»,3.688,3.594,3.688,"1,438","Мичуринский пр-т, д. 6; м. «Университет»"
7,8,ОАО «Медицина»,4.094,4.563,3.706,"3,313","Последний пер., д. 28"
8,40,ООО «Медицинская клиника «Семейный доктор»,3.786,3.714,3.714,"2,000","ул. Цандера, 5; м. «ВДНХ»"
9,23,ГУЗ г. Москвы «Городская поликлиника №220» Деп...,3.531,3.750,3.719,"1,750","Грохольский пер., д. 31; м. «Проспект Мира»"


In [8]:
d = d.drop([12,14])

In [9]:
d

,Unnamed: 0,1,3,4,5,6,adress
0,25,ЗАО «Группа компаний «Медси»,2.813,4.000,3.125,"1,375","2-й Тверской-Ямской пер., д. 10; м. «Маяковская»"
1,59,ООО «Управляющая компания «Медсервис»,3.417,3.333,3.500,"1,533","Мичуринский пр-т, д. 6"
2,22,ООО «Дирекция» (Поликлиника РУ),3.867,3.600,3.533,"2,071","Романов пер., д. 2; м. «Библиотека им. Ленина»"
3,29,ОАО «Медицина»,4.344,4.438,3.644,"3,143","ул. Сущевский вал, д. 12"
4,16,ООО «Клиника ЛМС» (Будь здоров),3.875,4.063,3.656,"2,563","ул. Тимура Фрунзе, 15/1; м. «Парк Культуры»"
5,35,ООО «Клиника ЛМС» (Будь здоров),3.929,4.143,3.679,"2,714","ул. Усачева, д. 33, стр. 3"
6,24,ЗАО «КООПВНЕШТОРГ»,3.688,3.594,3.688,"1,438","Мичуринский пр-т, д. 6; м. «Университет»"
7,8,ОАО «Медицина»,4.094,4.563,3.706,"3,313","Последний пер., д. 28"
8,40,ООО «Медицинская клиника «Семейный доктор»,3.786,3.714,3.714,"2,000","ул. Цандера, 5; м. «ВДНХ»"
9,23,ГУЗ г. Москвы «Городская поликлиника №220» Деп...,3.531,3.750,3.719,"1,750","Грохольский пер., д. 31; м. «Проспект Мира»"


In [107]:
f = open("Adresses.txt", 'r')
links = f.read()
f.close()

In [108]:
links = links.split('\n')

In [109]:
comments = []
for link in links:
    comments.append(get_all_comments(link))
    time.sleep(2)
print(comments)

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200

## Теперь, используя сервис indico.io, выделим эмоциональный окрас из полученных комментариев

In [113]:
indicoio.config.api_key = 'f4145fca4178193e8d4b499ac6122124'

In [126]:
sems = []
for klinic in comments:
    klinic_sems = []
    for com in klinic:
        klinic_sems.append(indicoio.sentiment(com, language='ru'))
    sems.append(klinic_sems)

In [158]:
sement = []
for ar in sems:
    sement.append(np.average(ar))

In [160]:
d['sem'] = sement

In [161]:
d

,Unnamed: 0,1,3,4,5,6,adress,sem
0,25,ЗАО «Группа компаний «Медси»,2.813,4.000,3.125,"1,375","2-й Тверской-Ямской пер., д. 10; м. «Маяковская»",0.495393
1,59,ООО «Управляющая компания «Медсервис»,3.417,3.333,3.500,"1,533","Мичуринский пр-т, д. 6",0.293387
2,22,ООО «Дирекция» (Поликлиника РУ),3.867,3.600,3.533,"2,071","Романов пер., д. 2; м. «Библиотека им. Ленина»",0.518172
3,29,ОАО «Медицина»,4.344,4.438,3.644,"3,143","ул. Сущевский вал, д. 12",0.487265
4,16,ООО «Клиника ЛМС» (Будь здоров),3.875,4.063,3.656,"2,563","ул. Тимура Фрунзе, 15/1; м. «Парк Культуры»",0.462602
5,35,ООО «Клиника ЛМС» (Будь здоров),3.929,4.143,3.679,"2,714","ул. Усачева, д. 33, стр. 3",0.817823
6,24,ЗАО «КООПВНЕШТОРГ»,3.688,3.594,3.688,"1,438","Мичуринский пр-т, д. 6; м. «Университет»",0.586886
7,8,ОАО «Медицина»,4.094,4.563,3.706,"3,313","Последний пер., д. 28",0.617936
8,40,ООО «Медицинская клиника «Семейный доктор»,3.786,3.714,3.714,"2,000","ул. Цандера, 5; м. «ВДНХ»",0.449647
9,23,ГУЗ г. Москвы «Городская поликлиника №220» Деп...,3.531,3.750,3.719,"1,750","Грохольский пер., д. 31; м. «Проспект Мира»",0.524562


In [163]:
d.to_csv('lpu_final.csv', encoding='UTF-8')